In [9]:
from keras.models import Model, Input, save_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras import losses
import h5py

DATA_NEIGHBORHOOD_FILE = 'data.h5'
MODEL_NEIGHBORHOOD = 'model.h5'

In [10]:
with h5py.File(DATA_NEIGHBORHOOD_FILE, 'r') as f:
    
    images_training = list(f['images_training'])
    steering_training = list(f['steering_training'])
    labels_training = list(f['label_training'])
        
    images_testing = list(f['images_testing'])
    steering_testing = list(f['steering_testing'])
    labels_testing = list(f['label_testing'])

In [3]:
activation = 'relu'

input_shape = (128, 128, 3)
pic_input = Input(shape=input_shape)

img_stack = Conv2D(64, (3, 3), name='convolution0', padding='same', activation=activation)(pic_input)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(64, (3, 3), name='convolution1', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(32, (3, 3), name='convolution2', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(32, (3, 3), name='convolution3', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Flatten()(img_stack)
img_stack = Dropout(0.25)(img_stack)

dense_stack = Dense(64, activation=activation, name='dense0')(img_stack)
dense_stack = Dropout(0.25)(dense_stack)
dense_stack = Dense(32, activation=activation, name='dense1')(dense_stack)
dense_stack = Dense(1, name='output', activation='tanh')(dense_stack)

adam = Adam()
model = Model(inputs=[pic_input], outputs=dense_stack)
model.compile(optimizer=adam, loss='mse')


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
convolution0 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
convolution1 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
convolution2 (Conv2D)        (None, 32, 32, 32)        18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [8]:
from generator import DriveDataGenerator

plateau_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
model_checkpoint = ModelCheckpoint(MODEL_LANDSCAPE, monitor='val_loss', save_best_only=True)
tensorboard_callback = TensorBoard(log_dir='./logs')

resize_dims = (input_shape[0], input_shape[1])
train_generator = DriveDataGenerator(images_training, labels_training, resize_dims=resize_dims)
validation_generator = DriveDataGenerator(images_testing, labels_testing, resize_dims=resize_dims)

model.fit_generator(generator=train_generator, validation_data=validation_generator,
                    epochs=500, verbose=1, 
                    callbacks=[model_checkpoint, tensorboard_callback])

Epoch 1/500
28/28 [==============================] - 8s 276ms/step - loss: 1.2145 - val_loss: 1.2517
Epoch 2/500
28/28 [==============================] - 3s 100ms/step - loss: 1.2645 - val_loss: 1.2437
Epoch 3/500
28/28 [==============================] - 3s 101ms/step - loss: 1.2932 - val_loss: 1.3299
Epoch 4/500
28/28 [==============================] - 3s 100ms/step - loss: 1.2928 - val_loss: 1.2619
Epoch 5/500
28/28 [==============================] - 3s 100ms/step - loss: 1.3008 - val_loss: 1.3180
Epoch 6/500
28/28 [==============================] - 3s 100ms/step - loss: 1.2286 - val_loss: 1.2830
Epoch 7/500
28/28 [==============================] - 3s 101ms/step - loss: 1.2693 - val_loss: 1.1233
Epoch 8/500
28/28 [==============================] - 3s 103ms/step - loss: 1.2369 - val_loss: 1.3054
Epoch 9/500
28/28 [==============================] - 3s 99ms/step - loss: 1.2836 - val_loss: 1.2034
Epoch 10/500
28/28 [==============================] - 3s 100ms/step - loss: 1.2752 - val_los

28/28 [==============================] - 3s 111ms/step - loss: 1.2185 - val_loss: 1.3551
Epoch 82/500
28/28 [==============================] - 3s 109ms/step - loss: 1.2630 - val_loss: 1.2272
Epoch 83/500
28/28 [==============================] - 3s 116ms/step - loss: 1.2477 - val_loss: 1.2217
Epoch 84/500
28/28 [==============================] - 3s 110ms/step - loss: 1.2377 - val_loss: 1.2641
Epoch 85/500
28/28 [==============================] - 3s 116ms/step - loss: 1.2710 - val_loss: 1.2825
Epoch 86/500
28/28 [==============================] - 3s 111ms/step - loss: 1.2120 - val_loss: 1.3166
Epoch 87/500
28/28 [==============================] - 3s 113ms/step - loss: 1.2771 - val_loss: 1.1666
Epoch 88/500
28/28 [==============================] - 4s 127ms/step - loss: 1.3013 - val_loss: 1.2203
Epoch 89/500
28/28 [==============================] - 3s 107ms/step - loss: 1.2998 - val_loss: 1.2179
Epoch 90/500
28/28 [==============================] - 3s 109ms/step - loss: 1.1942 - val_loss: 

Epoch 161/500
28/28 [==============================] - 3s 103ms/step - loss: 1.2367 - val_loss: 1.3301
Epoch 162/500
28/28 [==============================] - 3s 101ms/step - loss: 1.2940 - val_loss: 1.2944
Epoch 163/500
28/28 [==============================] - 3s 99ms/step - loss: 1.2274 - val_loss: 1.2616
Epoch 164/500
28/28 [==============================] - 3s 100ms/step - loss: 1.3351 - val_loss: 1.3043
Epoch 165/500
28/28 [==============================] - 3s 100ms/step - loss: 1.2446 - val_loss: 1.2279
Epoch 166/500
28/28 [==============================] - 3s 100ms/step - loss: 1.1945 - val_loss: 1.2934
Epoch 167/500
28/28 [==============================] - 3s 100ms/step - loss: 1.2779 - val_loss: 1.3741
Epoch 168/500
28/28 [==============================] - 3s 99ms/step - loss: 1.2959 - val_loss: 1.3426
Epoch 169/500
28/28 [==============================] - 3s 100ms/step - loss: 1.2720 - val_loss: 1.2358
Epoch 170/500
28/28 [==============================] - 3s 99ms/step - loss:

28/28 [==============================] - 2s 78ms/step - loss: 1.2725 - val_loss: 1.2409
Epoch 242/500
28/28 [==============================] - 2s 78ms/step - loss: 1.2636 - val_loss: 1.3459
Epoch 243/500
28/28 [==============================] - 2s 85ms/step - loss: 1.3090 - val_loss: 1.2743
Epoch 244/500
28/28 [==============================] - 2s 85ms/step - loss: 1.2880 - val_loss: 1.2950
Epoch 245/500
28/28 [==============================] - 2s 84ms/step - loss: 1.2104 - val_loss: 1.2655
Epoch 246/500
28/28 [==============================] - 3s 93ms/step - loss: 1.2057 - val_loss: 1.2881
Epoch 247/500
28/28 [==============================] - 3s 91ms/step - loss: 1.2223 - val_loss: 1.3002
Epoch 248/500
28/28 [==============================] - 2s 86ms/step - loss: 1.2682 - val_loss: 1.3242
Epoch 249/500
28/28 [==============================] - 2s 86ms/step - loss: 1.2321 - val_loss: 1.2689
Epoch 250/500
28/28 [==============================] - 2s 87ms/step - loss: 1.3362 - val_loss: 1

28/28 [==============================] - 3s 94ms/step - loss: 1.2870 - val_loss: 1.2585
Epoch 322/500
28/28 [==============================] - 3s 103ms/step - loss: 1.2347 - val_loss: 1.2930
Epoch 323/500
28/28 [==============================] - 3s 93ms/step - loss: 1.2205 - val_loss: 1.3152
Epoch 324/500
28/28 [==============================] - 3s 97ms/step - loss: 1.2421 - val_loss: 1.2058
Epoch 325/500
28/28 [==============================] - 3s 90ms/step - loss: 1.2601 - val_loss: 1.1728
Epoch 326/500
28/28 [==============================] - 3s 93ms/step - loss: 1.2679 - val_loss: 1.2689
Epoch 327/500
28/28 [==============================] - 2s 88ms/step - loss: 1.2723 - val_loss: 1.3112
Epoch 328/500
28/28 [==============================] - 2s 87ms/step - loss: 1.2400 - val_loss: 1.3235
Epoch 329/500
28/28 [==============================] - 2s 87ms/step - loss: 1.2362 - val_loss: 1.3188
Epoch 330/500
28/28 [==============================] - 2s 88ms/step - loss: 1.2681 - val_loss: 

28/28 [==============================] - 2s 81ms/step - loss: 1.2616 - val_loss: 1.2816
Epoch 402/500
28/28 [==============================] - 2s 80ms/step - loss: 1.2855 - val_loss: 1.3217
Epoch 403/500
28/28 [==============================] - 2s 81ms/step - loss: 1.2569 - val_loss: 1.1435
Epoch 404/500
28/28 [==============================] - 2s 83ms/step - loss: 1.2555 - val_loss: 1.2846
Epoch 405/500
28/28 [==============================] - 3s 94ms/step - loss: 1.2902 - val_loss: 1.1621
Epoch 406/500
28/28 [==============================] - 2s 84ms/step - loss: 1.2450 - val_loss: 1.2819
Epoch 407/500
28/28 [==============================] - 2s 86ms/step - loss: 1.2405 - val_loss: 1.2548
Epoch 408/500
28/28 [==============================] - 3s 92ms/step - loss: 1.2616 - val_loss: 1.2954
Epoch 409/500
28/28 [==============================] - 3s 89ms/step - loss: 1.2676 - val_loss: 1.2916
Epoch 410/500
28/28 [==============================] - 3s 89ms/step - loss: 1.2773 - val_loss: 1

28/28 [==============================] - 2s 79ms/step - loss: 1.2593 - val_loss: 1.1896
Epoch 482/500
28/28 [==============================] - 2s 81ms/step - loss: 1.2956 - val_loss: 1.2398
Epoch 483/500
28/28 [==============================] - 2s 80ms/step - loss: 1.2676 - val_loss: 1.2881
Epoch 484/500
28/28 [==============================] - 2s 82ms/step - loss: 1.2583 - val_loss: 1.1928
Epoch 485/500
28/28 [==============================] - 2s 81ms/step - loss: 1.2392 - val_loss: 1.2540
Epoch 486/500
28/28 [==============================] - 2s 80ms/step - loss: 1.2521 - val_loss: 1.2663
Epoch 487/500
28/28 [==============================] - 2s 80ms/step - loss: 1.2606 - val_loss: 1.2691
Epoch 488/500
28/28 [==============================] - 2s 79ms/step - loss: 1.2040 - val_loss: 1.1816
Epoch 489/500
28/28 [==============================] - 2s 79ms/step - loss: 1.1944 - val_loss: 1.2987
Epoch 490/500
28/28 [==============================] - 2s 81ms/step - loss: 1.2549 - val_loss: 1